In [ ]:
from flask import Flask, request, send_file, render_template_string
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from io import BytesIO
from pyngrok import conf, ngrok
from hashlib import sha256

app = Flask(__name__)

# Gán mã ngrok token của bạn
conf.get_default().auth_token = "2x78kyYgoItDVATusfDXfO8EB92_7BsHuMDhvAYST2gRrDws2"

# Chuẩn hóa khóa AES về độ dài phù hợp: 16, 24 hoặc 32 bytes
def get_aes_key(key: str) -> bytes:
    return sha256(key.encode()).digest()[:32]  # Chuẩn hóa khóa 32 byte bằng SHA-256

# Mã hóa dữ liệu bằng AES
def encrypt_file(data: bytes, key: str) -> bytes:
    key_bytes = get_aes_key(key)
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv=b"1234567890abcdef")
    encrypted = cipher.encrypt(pad(data, AES.block_size))
    return encrypted

# Giải mã dữ liệu bằng AES
def decrypt_file(data: bytes, key: str) -> bytes:
    key_bytes = get_aes_key(key)
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv=b"1234567890abcdef")
    decrypted = unpad(cipher.decrypt(data), AES.block_size)
    return decrypted

# Giao diện HTML
HTML = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>AES Encryption/Decryption</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #fff0c1, #c1fff9);
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }

        .container {
            background: white;
            padding: 30px;
            border-radius: 18px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.15);
            width: 400px;
            animation: fadeIn 0.6s ease-in-out;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(30px); }
            to { opacity: 1; transform: translateY(0); }
        }

        h1 {
            text-align: center;
            color: #ff7a00; /* Orange tone */
            margin-bottom: 20px;
        }

        form {
            display: flex;
            flex-direction: column;
        }

        input[type="file"],
        input[type="text"],
        select {
            margin: 10px 0;
            padding: 10px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 10px;
            transition: all 0.3s ease;
        }

        input[type="text"]:focus,
        select:focus {
            border-color: #28c6b9; /* Teal */
            box-shadow: 0 0 6px #28c6b9;
            outline: none;
        }

        input[type="submit"] {
            margin-top: 15px;
            padding: 12px;
            font-size: 16px;
            font-weight: bold;
            background: linear-gradient(to right, #f7b733, #fc4a1a); /* Orange gradient */
            color: white;
            border: none;
            border-radius: 10px;
            cursor: pointer;
            transition: background 0.3s ease;
        }

        input[type="submit"]:hover {
            background: linear-gradient(to right, #fc4a1a, #f7b733); /* reverse gradient */
        }

        .error {
            color: #d9534f;
            font-weight: bold;
            text-align: center;
        }

        .success {
            color: #5cb85c;
            font-weight: bold;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>AES Encrypt/Decrypt</h1>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="post" enctype="multipart/form-data" onsubmit="return validateForm()">
            <input type="file" name="file" id="file" required>
            <input type="text" name="key" id="key" placeholder="Enter your secret key" required>
            <select name="action">
                <option value="encrypt">Encrypt</option>
                <option value="decrypt">Decrypt</option>
            </select>
            <input type="submit" value="Submit">
        </form>
        <p id="form-error" class="error" style="display:none;"></p>
    </div>

    <script>
        function validateForm() {
            const key = document.getElementById("key").value;
            const file = document.getElementById("file").value;
            const errorElement = document.getElementById("form-error");

            if (!file) {
                errorElement.textContent = "Please select a file.";
                errorElement.style.display = "block";
                return false;
            }

            if (key.length < 1) {
                errorElement.textContent = "Key cannot be empty.";
                errorElement.style.display = "block";
                return false;
            }

            errorElement.style.display = "none";
            return true;
        }
    </script>
</body>
</html>

'''

# Route chính
@app.route('/', methods=['GET', 'POST'])
def home():
    error = None
    if request.method == 'POST':
        try:
            f = request.files['file']
            key = request.form['key']
            action = request.form['action']
            data = f.read()

            if action == 'encrypt':
                result = encrypt_file(data, key)
                filename = f"encrypted_{f.filename}"
            else:
                result = decrypt_file(data, key)
                filename = f"decrypted_{f.filename}"

            buffer = BytesIO(result)
            buffer.seek(0)
            return send_file(buffer, as_attachment=True, download_name=filename)

        except Exception as e:
            error = f"Lỗi: {str(e)}"

    return render_template_string(HTML, error=error)

# Khởi chạy ngrok và server Flask
public_url = ngrok.connect(5000)
print(f"📌 Truy cập web tại: {public_url}")

app.run(port=5000, use_reloader=False)


📌 Truy cập web tại: NgrokTunnel: "https://4015-34-10-234-239.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/May/2025 04:09:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/May/2025 04:09:11] "GET /favicon.ico HTTP/1.1" 404 -


In [5]:
!pip install pycryptodome

In [6]:
!pip install flask pyngrok pycryptodome